In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.python.estimator.export.export_output import PredictOutput

tf.logging.set_verbosity(tf.logging.INFO)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

train_data = np.empty([0,32,32,3])
train_labels = np.empty([0])

for i in range(1,6):
    path = os.path.join(os.getcwd(), 'cifar-10-batches-py/data_batch_' + str(i))
    batch = unpickle(path)
    images = batch['data'].reshape(10000,3,32,32)
    train_data = np.append(train_data, np.transpose(images, (0, 2, 3, 1)), axis=0)
    labels = np.asarray(batch['labels'])
    train_labels = np.append(train_labels, labels)
    
path = os.path.join(os.getcwd(), 'cifar-10-batches-py/test_batch')
batch = unpickle(path)
images = batch['data'].reshape(10000,3,32,32)
eval_data = np.transpose(images, (0, 2, 3, 1))
eval_labels = np.asarray(batch['labels']).astype(dtype=np.int32)
train_labels = train_labels.astype(dtype=np.int32)

train_data= np.true_divide(train_data,256).astype(dtype=np.float32)
eval_data= np.true_divide(eval_data,256).astype(dtype=np.float32)

print('train_data shape:   ' + str(train_data.shape) + str(train_data.dtype))
print('train_labels shape: ' + str(train_labels.shape) + str(train_labels.dtype))

print('eval_data shape:    ' + str(eval_data.shape) + str(eval_data.dtype))
print('eval_labels shape:  ' + str(eval_labels.shape) + str(eval_labels.dtype))

train_data shape:   (50000, 32, 32, 3)float32
train_labels shape: (50000,)int32
eval_data shape:    (10000, 32, 32, 3)float32
eval_labels shape:  (10000,)int32


In [7]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=features["x"],
        filters=64,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2)

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=1,
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=2048, activation=tf.nn.relu)
    dense1 = tf.layers.dense(inputs=dense, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    export_outputs={"SIGNATURE_NAME": PredictOutput({
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="final_result")
    })}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions,export_outputs=export_outputs)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./logdir")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f507467ac10>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './logdir', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [9]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "final_result"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [10]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=400,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=5000,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./logdir/model.ckpt.
INFO:tensorflow:probabilities = [[0.09093838 0.07474568 0.11262541 ... 0.08478592 0.09247474 0.12171735]
 [0.08348222 0.06979718 0.11587894 ... 0.09028836 0.09783461 0.12686327]
 [0.08715239 0.07107955 0.10573023 ... 0.08288124 0.09354153 0.12179182]
 ...
 [0.09007095 0.09022811 0.10785512 ... 0.09628767 0.09808921 0.11336883]
 [0.09400174 0.08907729 0.106447   ... 0.07794267 0.09665128 0.11278551]
 [0.08303661 0.07984626 0.11395706 ... 0.08492019 0.09792578 0.12178058]]
INFO:tensorflow:loss = 2.323733, step = 0
INFO:tensorflow:probabilities = [[0.00834759 0.02080552 0.11942159 ... 0.08816682 0.00237777 0.01071104]
 [0.01435737 0.00947652 0.18423796 ... 0.06204465 0.00197091 0.00401636]


INFO:tensorflow:loss = 1.1302757, step = 700 (5.270 sec)
INFO:tensorflow:probabilities = [[0.01129352 0.8010656  0.00940483 ... 0.00055672 0.00218126 0.0812567 ]
 [0.16310355 0.19311205 0.0001071  ... 0.00101859 0.00023462 0.6424009 ]
 [0.01274477 0.0204487  0.03036833 ... 0.8596803  0.00062993 0.03612637]
 ...
 [0.01519875 0.00300067 0.0765923  ... 0.6947356  0.0018501  0.00601786]
 [0.12574981 0.00015771 0.00164699 ... 0.00055435 0.8678239  0.00086866]
 [0.6549681  0.02212218 0.17914702 ... 0.0426861  0.0154537  0.01269685]] (2.647 sec)
INFO:tensorflow:global_step/sec: 18.9687
INFO:tensorflow:probabilities = [[0.9704521  0.0007989  0.0188305  ... 0.00128361 0.00056596 0.00080584]
 [0.1968081  0.04582214 0.01174267 ... 0.01234206 0.6806858  0.00996449]
 [0.01912238 0.00043661 0.03613096 ... 0.01394837 0.00275617 0.00200391]
 ...
 [0.16844025 0.01738479 0.00512278 ... 0.6281674  0.00270047 0.14269733]
 [0.8489046  0.00257671 0.016987   ... 0.01314459 0.01633901 0.00332456]
 [0.7562056 

INFO:tensorflow:loss = 0.63817245, step = 1500 (5.267 sec)
INFO:tensorflow:probabilities = [[0.0027462  0.95544857 0.00659146 ... 0.01138931 0.00142452 0.01255796]
 [0.00215906 0.00022403 0.00460587 ... 0.9363619  0.00001166 0.00087401]
 [0.08916603 0.00013847 0.07751981 ... 0.00393623 0.00194131 0.00141466]
 ...
 [0.00000731 0.00000283 0.00241802 ... 0.00002966 0.00000017 0.00001962]
 [0.9461113  0.0164419  0.00422429 ... 0.00031333 0.00747776 0.01999194]
 [0.00424021 0.00004155 0.01334836 ... 0.88140005 0.00001748 0.00076619]] (2.630 sec)
INFO:tensorflow:global_step/sec: 18.9912
INFO:tensorflow:probabilities = [[0.00148369 0.000002   0.6060519  ... 0.00719264 0.00002113 0.00009044]
 [0.00019408 0.00005674 0.00720483 ... 0.0748867  0.00082943 0.00006816]
 [0.082734   0.00206358 0.00075951 ... 0.00026469 0.9092378  0.00276319]
 ...
 [0.7334527  0.05757407 0.05558718 ... 0.00398852 0.00826531 0.00172169]
 [0.08049168 0.03253638 0.0024306  ... 0.00027437 0.8621506  0.00420719]
 [0.000073

INFO:tensorflow:loss = 0.34771818, step = 2300 (5.254 sec)
INFO:tensorflow:probabilities = [[0.22896342 0.00098454 0.04398336 ... 0.03511871 0.42673674 0.00844068]
 [0.02290498 0.04946959 0.00002777 ... 0.00000082 0.7863615  0.14071214]
 [0.00037201 0.0000055  0.9779138  ... 0.00017705 0.00001776 0.00002774]
 ...
 [0.00013638 0.08714099 0.00007816 ... 0.00000505 0.00045159 0.91129667]
 [0.         0.00002129 0.00000003 ... 0.         0.00000014 0.9999759 ]
 [0.00002515 0.98193973 0.00000038 ... 0.00000042 0.00000589 0.01802823]] (2.625 sec)
INFO:tensorflow:global_step/sec: 19.0289
INFO:tensorflow:probabilities = [[0.00000232 0.00000011 0.01482782 ... 0.95853025 0.00000003 0.00000129]
 [0.00000024 0.         0.00001276 ... 0.9996495  0.         0.00000002]
 [0.03014887 0.00005004 0.02013113 ... 0.01419412 0.00244685 0.00068348]
 ...
 [0.9978131  0.00078716 0.00005812 ... 0.00067894 0.00036383 0.0002961 ]
 [0.00025102 0.0000013  0.9842315  ... 0.00400057 0.         0.00000001]
 [0.000179

INFO:tensorflow:loss = 0.2773213, step = 3100 (5.255 sec)
INFO:tensorflow:probabilities = [[0.00000006 0.         0.00292325 ... 0.00580612 0.00000016 0.        ]
 [0.00006951 0.00007502 0.00031498 ... 0.00036086 0.0002836  0.00005505]
 [0.00010353 0.00006463 0.00029493 ... 0.00000187 0.00078323 0.00002891]
 ...
 [0.00168044 0.00000012 0.30623078 ... 0.00007065 0.00000387 0.00028821]
 [0.00000001 0.9999982  0.00000004 ... 0.00000027 0.00000014 0.00000004]
 [0.00267354 0.00002488 0.00000001 ... 0.00000029 0.00050584 0.9967937 ]] (2.624 sec)
INFO:tensorflow:global_step/sec: 19.0187
INFO:tensorflow:probabilities = [[0.00000002 0.00000002 0.00003859 ... 0.01304219 0.00000007 0.00000817]
 [0.         0.         0.00000323 ... 0.99996126 0.         0.        ]
 [0.00000001 0.         0.01571184 ... 0.01544558 0.0000005  0.00000001]
 ...
 [0.32109544 0.00023415 0.5094194  ... 0.01301233 0.08204783 0.01099884]
 [0.00000483 0.978195   0.00135846 ... 0.00000012 0.0019194  0.00024515]
 [0.9960775

INFO:tensorflow:loss = 0.10252551, step = 3900 (5.253 sec)
INFO:tensorflow:probabilities = [[0.00194957 0.         0.00000285 ... 0.         0.99802834 0.        ]
 [0.         0.         0.00000001 ... 0.         0.         0.        ]
 [0.1330706  0.00157262 0.04376148 ... 0.00373835 0.00005873 0.00004423]
 ...
 [0.         0.00000084 0.00001002 ... 0.00000024 0.         0.00000402]
 [0.06043098 0.00008749 0.00237854 ... 0.00000087 0.9368756  0.00001993]
 [0.9738191  0.00001559 0.01507072 ... 0.00104872 0.00244795 0.00244784]] (2.632 sec)
INFO:tensorflow:global_step/sec: 19.0684
INFO:tensorflow:probabilities = [[0.00262751 0.00000774 0.9731574  ... 0.00195448 0.00010878 0.00004287]
 [0.         0.         0.00000017 ... 0.00000076 0.00000004 0.        ]
 [0.00000001 0.         0.00003259 ... 0.9999554  0.00000001 0.        ]
 ...
 [0.00000002 0.         0.00586593 ... 0.36201587 0.         0.00000545]
 [0.00001868 0.         0.00004886 ... 0.01021417 0.00000011 0.00000024]
 [0.000087

INFO:tensorflow:loss = 0.072522685, step = 4700 (5.260 sec)
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.00000008 ... 0.00021824 0.         0.        ]
 [0.         0.         0.00000958 ... 0.         0.         0.        ]
 ...
 [0.00000066 0.86968243 0.         ... 0.         0.01315348 0.11716351]
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.996064   0.         0.00000001 ... 0.         0.00393595 0.00000003]] (2.611 sec)
INFO:tensorflow:global_step/sec: 19.1374
INFO:tensorflow:probabilities = [[0.00021487 0.94066745 0.00004647 ... 0.00000007 0.0000751  0.05899571]
 [0.00000063 0.         0.00000111 ... 0.         0.00013692 0.99986124]
 [0.00063481 0.00003489 0.         ... 0.         0.99933004 0.00000028]
 ...
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.00000003 ... 0.         0.         0.        ]
 [0.99737

In [11]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-22-21:22:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logdir/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-22-21:22:23
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6112, global_step = 5000, loss = 2.622987
{'loss': 2.622987, 'global_step': 5000, 'accuracy': 0.6112}


In [ ]:
# Export the model to saved_model.pb file
def serving_input_receiver():
    inputs = {"x": tf.placeholder(dtype=tf.float32, shape=[None,28,28,1])}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

export_dir = mnist_classifier.export_savedmodel(
    export_dir_base = "./logdir",
    serving_input_receiver_fn = serving_input_receiver,
    as_text=False)

In [ ]:
# Evaluate a sample image and display it
import matplotlib.pyplot as plt
from tensorflow.contrib import predictor

%matplotlib inline

image = eval_data[10].reshape(1,28,28,1)

predict_fn = predictor.from_saved_model(export_dir)
predictions = predict_fn(
    {"x": image})

print("Number: ", predictions['classes'][0])

# To interpret the values as a 28x28 image, we need to reshape
# the numpy array, which is one dimensional.
plt.imshow(image.reshape(28, 28), cmap=plt.cm.Greys);